## Soulbound

[Official rules.](https://cubicle7games.com/blog/soulbound-how-tests-work)

[StackExchange question.](https://math.stackexchange.com/questions/4403443/probability-of-rolling-n-dice-that-are-each-are-greater-than-or-equal-to-x-w)

[Reddit question.](https://www.reddit.com/r/RPGdesign/comments/zkduz7/understanding_the_probability_of_rolling_a_number/)

* Roll a pool of d6.
* Each die >= the DN is a success.
* Focus points can be used to increase the value of a die by 1 each.

Which dice should be upgraded? Since each focus point can be distributed independently,
you can't go wrong by upgrading the dice that are the closest to the threshold first.
Therefore, we can consider the dice in descending order and upgrade dice greedily.

In [1]:
%pip install icepool

from icepool import d6, MultisetEvaluator, Order, UnsupportedOrder

class SoulboundEvaluator(MultisetEvaluator):
    def __init__(self, dn, focus):
        self._dn = dn
        self._focus = focus

    def initial_state(self, order, outcomes, size):
        if order > 0:
            raise UnsupportedOrder()
        return 0, 0
    
    def next_state(self, state, order, outcome, count):
        success, focus = state
        
        if outcome >= self._dn:
            success += count
        else:
            # Upgrade as many dice as possible to successes.
            focus_per_die = self._dn - outcome
            focus_dice = min(focus // focus_per_die, count)
            success += focus_dice
            focus -= focus_per_die * focus_dice
        return success, focus
    
    def final_outcome(self, final_state, order, outcomes, *sizes):
        # Return just the number of successes.
        success, focus = final_state
        return success

result = SoulboundEvaluator(dn=5, focus=3).evaluate(d6.pool(10))
print(result)
print('Chance of at least 4 successes:')
print(result >= 4)

Die with denominator 60466176

| Outcome | Quantity | Probability |
|--------:|---------:|------------:|
|       0 |  1048576 |   1.734153% |
|       1 |  5242880 |   8.670765% |
|       2 | 11796480 |  19.509221% |
|       3 | 15728640 |  26.012295% |
|       4 | 13762560 |  22.760758% |
|       5 |  8257536 |  13.656455% |
|       6 |  3440640 |   5.690190% |
|       7 |   983040 |   1.625768% |
|       8 |   184320 |   0.304832% |
|       9 |    20480 |   0.033870% |
|      10 |     1024 |   0.001694% |


Chance of at least 4 successes:
Die with denominator 60466176

| Outcome | Quantity | Probability |
|:--------|---------:|------------:|
| False   | 33816576 |  55.926434% |
| True    | 26649600 |  44.073566% |


